In [1]:
#Get Data
#Need Ticker, returns from past 5 years, current price, beta, ESG score, and sector
#Need to get data from Yahoo Finance, ESG, and Sector
import yfinance as yf
import requests
import json
import pandas as pd
import numpy as np
import os
import time
import datetime
import pandas_datareader as pdr
from pandas_datareader import DataReader
from pypfopt import expected_returns, risk_models
from itertools import combinations
import yesg


In [2]:

def get_sp500_tickers():
    # Get S&P 500 tickers from Wikipedia
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    sp500 = pd.read_html(url)
    sp500 = sp500[0]
    return sp500["Symbol"].tolist()


def get_sp500_data():
    # Get S&P 500 tickers
    sp500 = get_sp500_tickers()

    # Filter out any tickers that may not be valid
    valid_tickers = []
    for ticker in sp500:
        try:
            yf.Ticker(ticker).info
            valid_tickers.append(ticker)
        except:
            pass

    # Initialize lists to store data
    rows = []

    # Fetch data for valid tickers
    for ticker in valid_tickers:
        # Fetch data for each ticker
        ticker_data = yf.Ticker(ticker)

        # Get historical price data
        history = ticker_data.history(period="5y")

        # Get beta and sector
        info = ticker_data.info
        beta = info.get("beta", None)
        sector = info.get("sector", None)

        # Append each row with date, ticker, price, beta, sector, and ESG score
        for date, row in history.iterrows():
            rows.append({
                "Date": date,
                "Ticker": ticker,
                "Price": row["Close"],
                "Beta": beta,
                "Sector": sector,
            })

    # Create DataFrame
    df = pd.DataFrame(rows)

    return df

# Example usage
sp500_df = get_sp500_data()


BRK.B: No data found, symbol may be delisted
BF.B: No price data found, symbol may be delisted (period=5y)


In [3]:
sp500_df['Year'] = sp500_df['Date'].dt.year
sp500_df['Year'] = sp500_df['Year'].astype(int)

In [4]:
sp500_df['Ticker'].nunique()

501

In [5]:
esg_df = pd.read_csv("esg_scores.csv")

sp500_df['Year'] = sp500_df['Year'].astype(int) 
esg_df['year'] = esg_df['year'].astype(int)

sp500_df_lim23 = sp500_df[sp500_df['Year'] <= 2023]

sp500_df_w_scores = sp500_df_lim23.merge( 
                                    esg_df,
                                    left_on=['Year','Ticker'],
                                    right_on=['year','Company_Symbol'],
                                    how='left',
                                    validate="m:1"
)

sp500_df_w_scores.drop(columns=['year','Company_Symbol'], inplace=True)
sp500_df_w_scores = sp500_df_w_scores.dropna(subset=['Total-Score'])
sp500_df_w_scores 

,Date,Ticker,Price,Beta,Sector,Year,Total-Score,E-Score,S-Score,G-Score
1178,2019-04-29 00:00:00-04:00,AOS,53.028980,1.253,Industrials,2019,46.630833,43.87,41.02,58.906667
1179,2019-04-30 00:00:00-04:00,AOS,49.950428,1.253,Industrials,2019,46.630833,43.87,41.02,58.906667
1180,2019-05-01 00:00:00-04:00,AOS,49.883919,1.253,Industrials,2019,46.630833,43.87,41.02,58.906667
1181,2019-05-02 00:00:00-04:00,AOS,49.902924,1.253,Industrials,2019,46.630833,43.87,41.02,58.906667
1182,2019-05-03 00:00:00-04:00,AOS,50.967110,1.253,Industrials,2019,46.630833,43.87,41.02,58.906667
...,...,...,...,...,...,...,...,...,...,...
583195,2023-12-22 00:00:00-05:00,ZTS,194.979996,0.848,Healthcare,2023,18.770000,3.24,6.81,8.720000
583196,2023-12-26 00:00:00-05:00,ZTS,195.500000,0.848,Healthcare,2023,18.770000,3.24,6.81,8.720000
583197,2023-12-27 00:00:00-05:00,ZTS,196.899994,0.848,Healthcare,2023,18.770000,3.24,6.81,8.720000
583198,2023-12-28 00:00:00-05:00,ZTS,197.160004,0.848,Healthcare,2023,18.770000,3.24,6.81,8.720000


In [6]:
sp500_df_w_scores['Ticker'].nunique()

434

In [7]:
first_date = pd.to_datetime(sp500_df_w_scores.iloc[0]['Date']).date()
last_date = pd.to_datetime(sp500_df_w_scores.iloc[-1]['Date']).date()

print("First date:", first_date)
print("Last date:", last_date)

First date: 2019-04-29
Last date: 2023-12-29


In [8]:
risk_free_rate = DataReader("IRLTLT01USM156N", "fred", start= first_date, end= last_date)
risk_free_rate = risk_free_rate.iloc[-1]/100
risk_free_rate = risk_free_rate.item()
risk_free_rate

0.04019999999999999

In [9]:
prices_firm = sp500_df_w_scores[['Ticker', 'Price']]
prices_firm

,Ticker,Price
1178,AOS,53.028980
1179,AOS,49.950428
1180,AOS,49.883919
1181,AOS,49.902924
1182,AOS,50.967110
...,...,...
583195,ZTS,194.979996
583196,ZTS,195.500000
583197,ZTS,196.899994
583198,ZTS,197.160004


In [10]:
grouped_data = sp500_df_w_scores.groupby('Ticker')

# Initialize empty dictionaries to store expected returns and covariance matrices for each firm
e_returns_dict = {}
cov_matrix_dict = {}

# Loop over each group (firm) and calculate expected returns and covariance matrix
for ticker, group in grouped_data:
    # Calculate expected returns using CAPM for the current firm
    e_returns_dict[ticker] = expected_returns.capm_return(group['Price'], risk_free_rate=risk_free_rate)
# Convert dictionaries to DataFrame
e_returns_df = pd.DataFrame(e_returns_dict)
e_returns_df= e_returns_df.T
e_returns_df.columns = ['Expected Returns']




c:\Users\nicks\anaconda3\Lib\site-packages\pypfopt\expected_returns.py:240: RuntimeWarning: prices are not in a dataframe
  warnings.warn("prices are not in a dataframe", RuntimeWarning)
c:\Users\nicks\anaconda3\Lib\site-packages\pypfopt\expected_returns.py:240: RuntimeWarning: prices are not in a dataframe
  warnings.warn("prices are not in a dataframe", RuntimeWarning)
c:\Users\nicks\anaconda3\Lib\site-packages\pypfopt\expected_returns.py:240: RuntimeWarning: prices are not in a dataframe
  warnings.warn("prices are not in a dataframe", RuntimeWarning)
c:\Users\nicks\anaconda3\Lib\site-packages\pypfopt\expected_returns.py:240: RuntimeWarning: prices are not in a dataframe
  warnings.warn("prices are not in a dataframe", RuntimeWarning)
c:\Users\nicks\anaconda3\Lib\site-packages\pypfopt\expected_returns.py:240: RuntimeWarning: prices are not in a dataframe
  warnings.warn("prices are not in a dataframe", RuntimeWarning)
c:\Users\nicks\anaconda3\Lib\site-packages\pypfopt\expected_retur

In [11]:
e_returns_df

,Expected Returns
A,0.138672
AAL,-0.172779
AAPL,0.334534
ABBV,0.190075
ABT,0.085503
...,...
XOM,0.090955
XYL,0.076876
YUM,0.062359
ZBH,0.012318


In [12]:
# Calculate returns for each firm
sp500_df_w_scores['Return'] = sp500_df_w_scores.groupby('Ticker')['Price'].pct_change()

# Group the DataFrame by 'Ticker'
grouped_data = sp500_df_w_scores.groupby('Ticker')

# Get unique tickers
tickers = list(grouped_data.groups.keys())

# Initialize an empty DataFrame to store covariance matrix
cov_matrix_df = pd.DataFrame(index=tickers, columns=tickers)

# Loop over each pair of firms and calculate the covariance between their return series
for ticker1, ticker2 in combinations(tickers, 2):
    returns1 = grouped_data.get_group(ticker1)['Return'].dropna()
    returns2 = grouped_data.get_group(ticker2)['Return'].dropna()
    
    # Ensure that both return series have the same length
    min_length = min(len(returns1), len(returns2))
    returns1 = returns1[:min_length]
    returns2 = returns2[:min_length]
    
    # Calculate the covariance between the return series of ticker1 and ticker2
    covariance = np.cov(returns1, returns2)[0, 1]
    
    # Fill in the covariance matrix (since it's symmetric)
    cov_matrix_df.loc[ticker1, ticker2] = covariance
    cov_matrix_df.loc[ticker2, ticker1] = covariance

# Replace NaN values for firms compared against themselves with variance
for ticker in tickers:
    variance = np.var(grouped_data.get_group(ticker)['Return'].dropna())
    cov_matrix_df.loc[ticker, ticker] = variance

# Save the covariance matrix DataFrame as a CSV file
cov_matrix_df.to_csv('covariance_matrix_returns.csv')

# Display the covariance matrix DataFrame
print("Covariance Matrix:")
cov_matrix_df


Covariance Matrix:


,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WRK,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZTS
A,0.000354,0.000226,0.000209,0.000113,0.000191,0.000176,0.000208,0.000242,0.000242,0.000147,...,0.000137,-0.000014,0.000248,0.000238,0.000119,0.000135,0.000216,0.000135,0.000166,0.000214
AAL,0.000226,0.001553,0.000249,0.000103,0.000136,0.000345,0.000259,0.000208,0.00033,0.00027,...,0.000231,-0.000038,0.0005,0.00076,0.00008,0.000345,0.000355,0.000269,0.000351,0.000176
AAPL,0.000209,0.000249,0.000417,0.000111,0.000178,0.000174,0.000236,0.000326,0.000287,0.000147,...,0.000154,-0.000025,0.000274,0.000284,0.000133,0.000143,0.000208,0.000152,0.000166,0.000214
ABBV,0.000113,0.000103,0.000111,0.000259,0.00012,0.000128,0.000118,0.000114,0.000121,0.000101,...,0.000125,-0.000027,0.000143,0.000134,0.000089,0.000115,0.000107,0.000085,0.000116,0.000118
ABT,0.000191,0.000136,0.000178,0.00012,0.000281,0.000149,0.000176,0.000193,0.000183,0.00012,...,0.000086,-0.000009,0.000202,0.000132,0.000136,0.00009,0.00017,0.000125,0.000151,0.000184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XOM,0.000135,0.000345,0.000143,0.000115,0.00009,0.000239,0.000158,0.000108,0.000204,0.000239,...,0.000117,-0.000022,0.000295,0.000368,0.000095,0.00049,0.000199,0.00015,0.0002,0.000111
XYL,0.000216,0.000355,0.000208,0.000107,0.00017,0.000247,0.000229,0.000212,0.000265,0.000191,...,0.000156,-0.000016,0.000319,0.000309,0.00015,0.000199,0.000402,0.000176,0.0002,0.000181
YUM,0.000135,0.000269,0.000152,0.000085,0.000125,0.000185,0.000167,0.000143,0.000177,0.000132,...,0.000033,-0.000022,0.000245,0.000271,0.000118,0.00015,0.000176,0.000281,0.000185,0.000161
ZBH,0.000166,0.000351,0.000166,0.000116,0.000151,0.000213,0.000196,0.000162,0.000212,0.000154,...,0.00009,-0.000003,0.000263,0.000352,0.000103,0.0002,0.0002,0.000185,0.000404,0.000162


In [13]:
#Subset main data frame into smaller data frames for each year
def subset_year(stock_returns_list, start, end) :
    stock_returns_ticker = stock_returns_list[(stock_returns_list['Year'] >= start) & 
                                              (stock_returns_list['Year'] <= end)]
    return stock_returns_ticker


    #essentially integrate and modify this for just the subsetting part.

#modify to subset by year instead of calculate returns
def subset_by_year(df, year):
    df_subset = df[df['Year'] == year]
    return df_subset




In [14]:
#Output the expected returns to a csv file
e_returns_df.to_csv('expected_returns.csv')

#Output the the sp500 data with scores to a csv file
sp500_df_w_scores.to_csv('sp500_data_with_scores.csv')